<a href="https://colab.research.google.com/github/soumyadiptapete/GNN_practice/blob/main/GCN_conv_web_traffic_prediction_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#predict average monthly web traffice on the chameleon dataset for each webpage reprrseented by a anode. the node features represent the presence of the words
!pip install torch-geometric
from torch_geometric.datasets import WikipediaNetwork
import torch_geometric.transforms as T
dataset= WikipediaNetwork(root='.',name='chameleon', transform=T.RandomNodeSplit(num_val=200, num_test=500))
data=dataset[0]

In [19]:
print(f'is directed: {data.is_directed()}')
print(f'number of nodes {data.x.shape[0]}')
print(f'data has isolated nodes {data.has_isolated_nodes()}')
print(f'data has loop: {data.has_self_loops()}')

is directed: True
number of nodes 2277
data has isolated nodes False
data has loop: True


In [20]:
data.y

tensor([0, 3, 3,  ..., 0, 4, 0])

In [21]:
#dwonlaod zipfile and open target file
import urllib.request
import zipfile

url = 'https://snap.stanford.edu/data/wikipedia.zip'
filehandle, _ = urllib.request.urlretrieve(url)
zip_file_object = zipfile.ZipFile(filehandle, 'r')
print(zip_file_object.namelist())
file = zip_file_object.open(first_file)
content = file.read()

['wikipedia/', 'wikipedia/chameleon/', 'wikipedia/chameleon/musae_chameleon_edges.csv', 'wikipedia/chameleon/musae_chameleon_features.json', 'wikipedia/chameleon/musae_chameleon_target.csv', 'wikipedia/crocodile/', 'wikipedia/crocodile/musae_crocodile_edges.csv', 'wikipedia/crocodile/musae_crocodile_features.json', 'wikipedia/crocodile/musae_crocodile_target.csv', 'wikipedia/squirrel/', 'wikipedia/squirrel/musae_squirrel_edges.csv', 'wikipedia/squirrel/musae_squirrel_features.json', 'wikipedia/squirrel/musae_squirrel_target.csv', 'wikipedia/citing.txt', 'wikipedia/README.txt']


In [22]:
target_file=zip_file_object.namelist()[4]


In [23]:
# import monthly web traffic from a webpage and convert it to target
import pandas as pd
import numpy as np
import torch
df=pd.read_csv(zip_file_object.open(target_file))
values= np.log10(df['target'])
data.y=torch.tensor(values)

In [49]:
from torch_geometric.nn import GCNConv
from torch.nn import Linear
import torch.nn.functional as F

#create vanilla GNN
class GNN(torch.nn.Module):
  def __init__(self, dim_in,dim_h,dim_out):
    super().__init__()
    self.GNN1=GCNConv(dim_in,dim_h**4)
    self.GNN2=GCNConv(dim_h**4,dim_h**2)
    self.GNN3=GCNConv(dim_h**2,dim_h)
    self.linear=Linear(dim_h,dim_out)

  def forward(self,x,edge_index):
    x=self.GNN1(x,edge_index)#N*dim_h
    x=F.relu(x)
    x=F.dropout(x,p=0.5)
    x=self.GNN2(x,edge_index)#N*dim_out
    x=F.relu(x)
    x=F.dropout(x,p=0.5)
    x=self.GNN3(x,edge_index)
    x=self.linear(x)
    return x.reshape(-1)


  def fit(self,data,epochs):
    criterion=torch.nn.MSELoss()
    optimizer=torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)

    y=data.y#labels


    for epoch in range(epochs+1):
      out=self(data.x,data.edge_index)#output of network forward pass
      optimizer.zero_grad()
      loss=criterion(out[data.train_mask],y[data.train_mask].float())
      # print train loss and validation loss
      if epoch%20==0:
        val_loss=criterion(out[data.val_mask],y[data.val_mask])

        print(f'Train Loss : {loss}')
        print(f'Validation loss: {val_loss}')


      loss.backward()
      optimizer.step()

  @torch.no_grad()
  def test(self,data):
    self.eval()
    out=self(data.x,data.edge_index)
    y_pred= out[data.test_mask]
    y_actual=data.y[data.test_mask]
    mse_test=F.mse_loss(y_pred,y_actual)
    print(f'Test MSE : {mse_test}')

In [50]:
model=GNN(data.x.shape[1], 4, 1)
model

GNN(
  (GNN1): GCNConv(2325, 256)
  (GNN2): GCNConv(256, 16)
  (GNN3): GCNConv(16, 4)
  (linear): Linear(in_features=4, out_features=1, bias=True)
)

In [51]:
out=model(data.x,data.edge_index)
print(out.shape)
print(data.y.shape)

torch.Size([2277])
torch.Size([2277])


In [52]:
model.fit(data,100)


Train Loss : 11.285555839538574
Validation loss: 12.062327686696841
Train Loss : 4.538243770599365
Validation loss: 5.136261933357084
Train Loss : 3.3488004207611084
Validation loss: 4.70721158802647
Train Loss : 2.425440549850464
Validation loss: 3.9853792418754033
Train Loss : 1.5200341939926147
Validation loss: 3.3479700577522133
Train Loss : 0.8074051141738892
Validation loss: 3.8952602254462096
